# T2 Decay Measurement Notebook
This notebook compares T2 decay extracted from three pulse sequences: Hahn Echo, Carr-Purcell (CP), and Carr-Purcell-Meiboom-Gill (CPMG).

## Imports

In [109]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import hilbert
from scipy.optimize import curve_fit
import os


## Load Data

In [110]:
import re

# Assume your data files are in subfolders named 'hahn', 'cp', and 'cpmg'
def load_data(folder, prefix):
    data = []
    pulse_times = []
    for fname in sorted(os.listdir(folder)):
        # Check that file is CSV and is of the form "PREFIXNUMBER.csv"
        if fname.endswith('.csv') and re.match(rf'^{prefix}\d+\.csv$', fname):
            pulse_time = 50
            df = pd.read_csv(os.path.join(folder, fname), header=None, names=['t', 'CH1', 'CH2']).dropna()
            df = df[(df['t'] > 0.007) & (df['t'] < 0.042)]
            df['CH1'] = -(df['CH1'] - df['CH1'].mean())
            df['CH2'] = df['CH2'] - df['CH2'].mean()

            # Subtract linear fit
            x = df['t']
            y = df['CH1']
            coeffs = np.polyfit(x, y, 1)
            linear_fit = np.polyval(coeffs, x)
            df['CH1'] -= linear_fit

            # Subract linear fit from CH2
            y = df['CH2']
            coeffs = np.polyfit(x, y, 1)
            linear_fit = np.polyval(coeffs, x)
            df['CH2'] -= linear_fit
            
            data.append(df)
            pulse_times.append(pulse_time)
    return data, pulse_times

# hahn_data, hahn_times = load_data('327', 'HAHN')
cp_data, cp_times = load_data('327', "CP")
cpmg_data, cpmg_times = load_data('327', "CPMG")


## Model Definition

In [111]:

def model_exp(t, A, T2, C):
    return A * np.exp(-t / T2) + C

def fit_decay(df, prominence=0.2, min_distance=100):
    t = df['t'].values
    real_signal = df['CH1'].values
    peaks, _ = find_peaks(real_signal, prominence=prominence, distance=min_distance)
    t_peaks = t[peaks]
    amp_peaks = real_signal[peaks]
    if len(t_peaks) < 3:
        return [np.nan, np.nan, np.nan], t_peaks, amp_peaks
    p0 = [amp_peaks[0], (t.max() - t.min()) / 2, np.min(amp_peaks)]
    popt, _ = curve_fit(model_exp, t_peaks, amp_peaks, p0=p0)
    return popt, t_peaks, amp_peaks


## Robust Echo Detection Using Hilbert Envelope
To avoid pulse artifacts and extract clean echo peaks, we use the Hilbert transform to compute the envelope of the signal. This lets us find only the main echo peaks, and ignore the high-frequency oscillations within each echo and the large transient artifacts near the pulses.

In [112]:

def fit_decay_envelope(df, channel='CH1', prominence=0.2, min_distance=150, pulse_threshold=0.95, left_mask_window=125, right_mask_window=125):
    t = df['t'].values
    signal_raw = df[channel].values
    
    # 1. Min-max scale the signal
    scaled = (signal_raw - np.min(signal_raw)) / (np.max(signal_raw) - np.min(signal_raw))

    # 2. Detect and mask where scaled signal > 0.95
    mask = np.ones_like(signal_raw, dtype=bool)
    pulse_locs = np.where(scaled > pulse_threshold)[0]
    for p in pulse_locs:
        mask[max(0, p - left_mask_window):min(len(mask), p + right_mask_window)] = False

    
    # 2. Compute Hilbert envelope
    analytic_signal = hilbert(signal_raw)
    from scipy.ndimage import gaussian_filter1d

    envelope = np.abs(analytic_signal)

    # Smooth envelope using Gaussian filter
    envelope = gaussian_filter1d(envelope, sigma=3)

    envelope[~mask] = 0



    
    # 3. Find peaks in the envelope where not masked
    t_masked = t[mask]
    env_masked = envelope[mask]
    peak_indices, _ = find_peaks(env_masked, prominence=prominence, distance=min_distance)
    t_peaks = t_masked[peak_indices]
    amp_peaks = env_masked[peak_indices]
    
    if len(t_peaks) < 3:
        return [np.nan, np.nan, np.nan], t_peaks, amp_peaks, envelope
    p0 = [amp_peaks[0], (t_peaks[-1] - t_peaks[0]) / 2, amp_peaks[-1]]
    popt, _ = curve_fit(model_exp, t_peaks, amp_peaks, p0=p0, maxfev=10000)
    return popt, t_peaks, amp_peaks, envelope


In [113]:
def analyze_with_envelope(data, pulse_times):
    results = []
    all_fits = []

    for idx, df in enumerate(data):
        # Real (CH1)
        popt_real, t_peaks_r, amp_peaks_r, env_r = fit_decay_envelope(df, channel='CH1', prominence=0.2)
        # Imaginary (CH2)
        popt_imag, t_peaks_i, amp_peaks_i, env_i = fit_decay_envelope(df, channel='CH2', prominence=0.2)

        results.append({
            'Pulse Time (us)': pulse_times[idx],
            'T2_Real (us)': popt_real[1],
            'T2_Imag (us)': popt_imag[1],
            'A_Real': popt_real[0],
            'A_Imag': popt_imag[0],
            'C_Real': popt_real[2],
            'C_Imag': popt_imag[2],
        })

        all_fits.append({
            't': df['t'].values,
            'CH1': df['CH1'].values,
            'CH2': df['CH2'].values,
            'Envelope_CH1': env_r,
            'Envelope_CH2': env_i,
            't_peaks_CH1': t_peaks_r,
            't_peaks_CH2': t_peaks_i,
            'amp_peaks_CH1': amp_peaks_r,
            'amp_peaks_CH2': amp_peaks_i,
            'fit_CH1': popt_real,
            'fit_CH2': popt_imag,
        })

    return pd.DataFrame(results), all_fits


# hahn_results, hahn_fits = analyze_with_envelope(hahn_data, hahn_times)
cp_results, cp_fits = analyze_with_envelope(cp_data, cp_times)
cpmg_results, cpmg_fits = analyze_with_envelope(cpmg_data, cpmg_times)

# display(hahn_results)
display(cp_results)
display(cpmg_results)


,Pulse Time (us),T2_Real (us),T2_Imag (us),A_Real,A_Imag,C_Real,C_Imag
0,50,0.008147,0.008084,5.548567,5.447398,0.500720,0.425958
1,50,0.007171,0.007362,5.810072,5.572705,0.502907,0.401658
2,50,0.007593,0.006932,6.093468,6.615990,0.480388,0.392515
3,50,0.007238,0.008111,5.854949,4.906974,0.569407,0.444649
4,50,0.008339,0.008725,5.140555,4.617465,0.522819,0.431616
5,50,0.007836,0.007898,5.502103,5.212375,0.515898,0.458837
6,50,0.007280,0.006945,5.898867,6.364870,0.539761,0.427889
7,50,0.007103,0.007184,6.143740,6.083714,0.530131,0.449597
8,50,0.007773,0.008219,5.721170,5.055063,0.543724,0.450750
9,50,0.008368,0.008104,5.161958,5.428220,0.509316,0.421806


,Pulse Time (us),T2_Real (us),T2_Imag (us),A_Real,A_Imag,C_Real,C_Imag
0,50,0.021291,0.023449,3.062263,2.817965,0.147734,0.116431
1,50,0.021796,0.023892,2.988155,2.669323,0.179450,0.180019
2,50,0.027711,0.026504,2.986546,2.770091,0.012965,0.082076
3,50,0.034885,0.024853,3.210494,2.786139,-0.363486,0.113462
4,50,0.026893,0.027622,2.915038,2.796829,0.035303,0.049925
5,50,0.027658,0.024751,3.054762,2.757948,-0.038224,0.129836
6,50,0.021154,0.024426,2.948863,2.861876,0.220965,0.072463
7,50,0.021751,0.021354,2.947468,2.780096,0.188943,0.235630
8,50,0.025662,0.025053,2.885263,2.808019,0.077817,0.100247
9,50,0.017694,0.019549,3.102698,2.885473,0.349465,0.272058


In [114]:
# for i, fit in enumerate(cp_fits):  # plot a few examples
#     t = fit['t']
    
#     # CH1 (Real)
#     raw1 = fit['CH1']
#     env1 = fit['Envelope_CH1']
#     t_peaks1 = fit['t_peaks_CH1']
#     amp_peaks1 = fit['amp_peaks_CH1']
#     popt1 = fit['fit_CH1']

#     # CH2 (Imag)
#     raw2 = fit['CH2']
#     env2 = fit['Envelope_CH2']
#     t_peaks2 = fit['t_peaks_CH2']
#     amp_peaks2 = fit['amp_peaks_CH2']
#     popt2 = fit['fit_CH2']
    
#     plt.figure(figsize=(14, 4))

#     # --- CH1 (Real) ---
#     plt.subplot(1, 2, 1)
#     plt.plot(t, raw1, 'k-', alpha=0.3, label='Raw CH1')
#     plt.plot(t, env1, 'orange', label='Envelope')
#     plt.plot(t_peaks1, amp_peaks1, 'ro', label='Echo Peaks')
#     if not np.isnan(popt1).any():
#         plt.plot(t, model_exp(t, *popt1), 'b--', label='Exp Fit')
#     plt.title(f'Dataset {i+1} - CH1 (Real)')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     # --- CH2 (Imag) ---
#     plt.subplot(1, 2, 2)
#     plt.plot(t, raw2, 'k-', alpha=0.3, label='Raw CH2')
#     plt.plot(t, env2, 'orange', label='Envelope')
#     plt.plot(t_peaks2, amp_peaks2, 'ro', label='Echo Peaks')
#     if not np.isnan(popt2).any():
#         plt.plot(t, model_exp(t, *popt2), 'b--', label='Exp Fit')
#     plt.title(f'Dataset {i+1} - CH2 (Imag)')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     plt.tight_layout()
#     plt.show()

In [115]:
# --- Summary statistics for T2 ---
t2_real = cp_results['T2_Real (us)'].dropna() / 1e-3
t2_imag = cp_results['T2_Imag (us)'].dropna() / 1e-3

def summarize(series, label):
    mean = series.mean()
    std = series.std()
    sem = std / np.sqrt(len(series))
    ci95 = 1.96 * sem
    print(f"{label}:\n  Mean T2 = {mean:.4f} ms\n  Std = {std:.4f}\n  SEM = {sem:.4f}\n  95% CI = ±{ci95:.4f}\n")

print("--- CP RESULTS ---")
summarize(t2_real, "CH1 (Real)")
summarize(t2_imag, "CH2 (Imag)")

# --- Summary statistics for T2 ---
t2_real = cpmg_results['T2_Real (us)'].dropna() / 1e-3
t2_imag = cpmg_results['T2_Imag (us)'].dropna() / 1e-3

def summarize(series, label):
    mean = series.mean()
    std = series.std()
    sem = std / np.sqrt(len(series))
    ci95 = 1.96 * sem
    print(f"{label}:\n  Mean T2 = {mean:.4f} ms\n  Std = {std:.4f}\n  SEM = {sem:.4f}\n  95% CI = ±{ci95:.4f}\n")

print("--- CPMG RESULTS ---")
summarize(t2_real, "CH1 (Real)")
summarize(t2_imag, "CH2 (Imag)")

--- CP RESULTS ---
CH1 (Real):
  Mean T2 = 7.6849 ms
  Std = 0.4846
  SEM = 0.1533
  95% CI = ±0.3004

CH2 (Imag):
  Mean T2 = 7.7564 ms
  Std = 0.6099
  SEM = 0.1929
  95% CI = ±0.3780

--- CPMG RESULTS ---
CH1 (Real):
  Mean T2 = 24.6494 ms
  Std = 4.9179
  SEM = 1.5552
  95% CI = ±3.0481

CH2 (Imag):
  Mean T2 = 24.1451 ms
  Std = 2.3306
  SEM = 0.7370
  95% CI = ±1.4445

